<a href="https://colab.research.google.com/github/Lili786965/python/blob/main/%D0%9D%D1%83%D1%80%D0%B5%D1%82%D0%B4%D0%B8%D0%BD%D0%BE%D0%B2%D0%B0%D0%9B%D0%B8%D0%BD%D0%B8%D0%B7%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Вариант 18. Учет и управление сервисом доставки еды

In [9]:
from abc import ABC, ABCMeta, abstractmethod
from dataclasses import dataclass, field
from typing import List, Dict, Type, Optional
import json
import logging
from enum import Enum

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('restaurant.log'),
        logging.StreamHandler()
    ]
)

class InvalidDishError(Exception):
    """Исключение для некорректных данных блюда"""
    pass

class PermissionDeniedError(Exception):
    """Исключение для отсутствия прав доступа"""
    pass

class OrderNotFoundError(Exception):
    """Исключение для отсутствия заказа"""
    pass

class Deliverable(ABC):
    @abstractmethod
    def deliver_order(self, order_id: int) -> str:
        pass

class Reportable(ABC):
    @abstractmethod
    def generate_report(self) -> str:
        pass

class LoggingMixin:
    def log_action(self, action: str):
        logging.info(f"{self.__class__.__name__}: {action}")

class NotificationMixin:
    def send_notification(self, message: str):
        print(f"Notification: {message}")

@dataclass
class Customer:
    name: str
    phone: str
    email: str = ""

class DishMeta(ABCMeta):
    def __new__(cls, name, bases, namespace):
        new_class = super().__new__(cls, name, bases, namespace)
        if name != 'Dish':
            Dish._registry[name.lower()] = new_class
        return new_class

class Dish(ABC, metaclass=DishMeta):
    _registry: Dict[str, Type['Dish']] = {}

    def __init__(self, dish_id: int, name: str, price: float, ingredients: List[str], is_available: bool = True):
        self._dish_id = dish_id
        self._name = name
        self._price = price
        self._ingredients = ingredients
        self._is_available = is_available

    @abstractmethod
    def calculate_cost(self) -> float:
        pass

    def __str__(self) -> str:
        return f"Блюдо: {self._name}, Цена: {self._price}"

    def __lt__(self, other) -> bool:
        return self._price < other._price

    def __gt__(self, other) -> bool:
        return self._price > other._price

    def __eq__(self, other) -> bool:
        return self._dish_id == other._dish_id

    @property
    def dish_id(self) -> int:
        return self._dish_id

    @property
    def name(self) -> str:
        return self._name

    @name.setter
    def name(self, value: str):
        if not value:
            raise InvalidDishError("Название блюда не может быть пустым")
        self._name = value

    @property
    def price(self) -> float:
        return self._price

    @price.setter
    def price(self, value: float):
        if value <= 0:
            raise InvalidDishError("Цена должна быть положительной")
        self._price = value

    @property
    def ingredients(self) -> List[str]:
        return self._ingredients

    @ingredients.setter
    def ingredients(self, value: List[str]):
        if not value:
            raise InvalidDishError("Список ингредиентов не может быть пустым")
        self._ingredients = value

    @property
    def is_available(self) -> bool:
        return self._is_available

    @is_available.setter
    def is_available(self, value: bool):
        self._is_available = value

    def to_dict(self) -> Dict:
        return {
            'type': self.__class__.__name__,
            'dish_id': self._dish_id,
            'name': self._name,
            'price': self._price,
            'ingredients': self._ingredients,
            'is_available': self._is_available
        }

    @classmethod
    def from_dict(cls, data: Dict) -> 'Dish':
        dish_type = data.pop('type')
        return DishFactory.create_dish(dish_type, **data)

class MainCourse(Dish, LoggingMixin):
    def __init__(self, dish_id: int, name: str, price: float, ingredients: List[str], calories: int, is_available: bool = True):
        super().__init__(dish_id, name, price, ingredients, is_available)
        self._calories = calories
        self.log_action(f"Создано основное блюдо: {name}")

    def calculate_cost(self) -> float:
        return self._price * (1.1 if self._calories > 500 else 1.0)

    def __str__(self) -> str:
        return f"Основное блюдо: {self._name}, Цена: {self._price}, Калорийность: {self._calories}"

    @property
    def calories(self) -> int:
        return self._calories

    @calories.setter
    def calories(self, value: int):
        if value <= 0:
            raise InvalidDishError("Калорийность должна быть положительной")
        self._calories = value

    def to_dict(self) -> Dict:
        data = super().to_dict()
        data['calories'] = self._calories
        return data

class Dessert(Dish, LoggingMixin):
    def __init__(self, dish_id: int, name: str, price: float, ingredients: List[str], sugar_content: float, is_available: bool = True):
        super().__init__(dish_id, name, price, ingredients, is_available)
        self._sugar_content = sugar_content
        self.log_action(f"Создан десерт: {name}")

    def calculate_cost(self) -> float:
        return self._price * (0.9 if self._sugar_content < 10 else 1.0)

    def __str__(self) -> str:
        return f"Десерт: {self._name}, Цена: {self._price}, Содержание сахара: {self._sugar_content}g"

    @property
    def sugar_content(self) -> float:
        return self._sugar_content

    @sugar_content.setter
    def sugar_content(self, value: float):
        if value < 0:
            raise InvalidDishError("Содержание сахара не может быть отрицательным")
        self._sugar_content = value

    def to_dict(self) -> Dict:
        data = super().to_dict()
        data['sugar_content'] = self._sugar_content
        return data

class Beverage(Dish, LoggingMixin):
    def __init__(self, dish_id: int, name: str, price: float, ingredients: List[str], volume: float, is_available: bool = True):
        super().__init__(dish_id, name, price, ingredients, is_available)
        self._volume = volume
        self.log_action(f"Создан напиток: {name}")

    def calculate_cost(self) -> float:
        return self._price * (0.8 if self._volume > 0.5 else 1.0)

    def __str__(self) -> str:
        return f"Напиток: {self._name}, Цена: {self._price}, Объем: {self._volume}л"

    @property
    def volume(self) -> float:
        return self._volume

    @volume.setter
    def volume(self, value: float):
        if value <= 0:
            raise InvalidDishError("Объем должен быть положительным")
        self._volume = value

    def to_dict(self) -> Dict:
        data = super().to_dict()
        data['volume'] = self._volume
        return data

class Order(LoggingMixin, NotificationMixin, Deliverable, Reportable):
    def __init__(self, order_id: int, customer: Customer, delivery_address: str = ""):
        self._order_id = order_id
        self._customer = customer
        self._dishes: List[Dish] = []
        self._delivery_address = delivery_address
        self._total_cost = 0.0
        self.log_action(f"Создан заказ #{order_id}")

    def add_dish(self, dish: Dish):
        if not dish.is_available:
            raise InvalidDishError(f"Блюдо {dish.name} недоступно")
        self._dishes.append(dish)
        self._total_cost = self.calculate_total()
        self.log_action(f"Добавлено блюдо {dish.name} в заказ #{self._order_id}")
        self.send_notification(f"В заказ #{self._order_id} добавлено блюдо: {dish.name}")

    def remove_dish(self, dish: Dish):
        if dish in self._dishes:
            self._dishes.remove(dish)
            self._total_cost = self.calculate_total()
            self.log_action(f"Удалено блюдо {dish.name} из заказа #{self._order_id}")
            self.send_notification(f"Из заказа #{self._order_id} удалено блюдо: {dish.name}")

    def calculate_total(self) -> float:
        return sum(dish.calculate_cost() for dish in self._dishes)

    def deliver_order(self, order_id: int) -> str:
        if order_id != self._order_id:
            raise OrderNotFoundError(f"Заказ #{order_id} не найден")
        self.log_action(f"Заказ #{order_id} доставлен по адресу: {self._delivery_address}")
        self.send_notification(f"Ваш заказ #{order_id} доставлен!")
        return f"Заказ #{order_id} доставлен"

    def generate_report(self) -> str:
        dishes_info = "\n".join(str(dish) for dish in self._dishes)
        return (
            f"Отчет по заказу #{self._order_id}\n"
            f"Клиент: {self._customer.name}\n"
            f"Телефон: {self._customer.phone}\n"
            f"Адрес доставки: {self._delivery_address}\n"
            f"Блюда:\n{dishes_info}\n"
            f"Общая стоимость: {self._total_cost}"
        )

    def to_dict(self) -> Dict:
        return {
            'order_id': self._order_id,
            'customer': {
                'name': self._customer.name,
                'phone': self._customer.phone,
                'email': self._customer.email
            },
            'dishes': [dish.to_dict() for dish in self._dishes],
            'delivery_address': self._delivery_address,
            'total_cost': self._total_cost
        }

    @classmethod
    def from_dict(cls, data: Dict) -> 'Order':
        customer_data = data['customer']
        customer = Customer(
            name=customer_data['name'],
            phone=customer_data['phone'],
            email=customer_data.get('email', '')
        )
        order = Order(data['order_id'], customer, data['delivery_address'])
        for dish_data in data['dishes']:
            dish_type = dish_data.pop('type')
            dish = DishFactory.create_dish(dish_type, **dish_data)
            order.add_dish(dish)
        return order

    @property
    def order_id(self) -> int:
        return self._order_id

    @property
    def customer(self) -> Customer:
        return self._customer

    @property
    def dishes(self) -> List[Dish]:
        return self._dishes

    @property
    def delivery_address(self) -> str:
        return self._delivery_address

    @delivery_address.setter
    def delivery_address(self, value: str):
        self._delivery_address = value

    @property
    def total_cost(self) -> float:
        return self._total_cost

class DishFactory:
    @staticmethod
    def create_dish(dish_type: str, **kwargs) -> Dish:
        dish_class = Dish._registry.get(dish_type.lower())
        if not dish_class:
            raise InvalidDishError(f"Неизвестный тип блюда: {dish_type}")
        return dish_class(**kwargs)

class OrderChangeHandler(ABC):
    def __init__(self, successor: Optional['OrderChangeHandler'] = None):
        self._successor = successor

    @abstractmethod
    def handle_request(self, order: Order, change_request: Dict) -> bool:
        pass

class CallCenterOperator(OrderChangeHandler):
    def handle_request(self, order: Order, change_request: Dict) -> bool:
        if change_request.get('change_type') == 'minor':
            print("Оператор колл-центра обработал незначительное изменение")
            return True
        elif self._successor:
            return self._successor.handle_request(order, change_request)
        return False

class Manager(OrderChangeHandler):
    def handle_request(self, order: Order, change_request: Dict) -> bool:
        if change_request.get('change_type') == 'price_change':
            print("Менеджер одобрил изменение стоимости")
            return True
        elif self._successor:
            return self._successor.handle_request(order, change_request)
        return False

class Admin(OrderChangeHandler):
    def handle_request(self, order: Order, change_request: Dict) -> bool:
        print("Администратор одобрил любое изменение")
        return True

class DeliveryProcess(ABC):
    def deliver_order(self, order: Order) -> str:
        self.prepare_order(order)
        self.deliver(order)
        return self.confirm_delivery(order)

    @abstractmethod
    def prepare_order(self, order: Order):
        pass

    @abstractmethod
    def deliver(self, order: Order):
        pass

    @abstractmethod
    def confirm_delivery(self, order: Order) -> str:
        pass

class CourierDeliveryProcess(DeliveryProcess):
    def prepare_order(self, order: Order):
        print(f"Подготовка заказа #{order.order_id} для курьерской доставки")

    def deliver(self, order: Order):
        print(f"Доставка заказа #{order.order_id} курьером")

    def confirm_delivery(self, order: Order) -> str:
        return f"Заказ #{order.order_id} доставлен курьером"

class SelfPickupProcess(DeliveryProcess):
    def prepare_order(self, order: Order):
        print(f"Подготовка заказа #{order.order_id} для самовывоза")

    def deliver(self, order: Order):
        print(f"Ожидание клиента для заказа #{order.order_id}")

    def confirm_delivery(self, order: Order) -> str:
        return f"Заказ #{order.order_id} выдан клиенту"

def check_permissions(required_role: str):
    def decorator(func):
        def wrapper(*args, **kwargs):
            user_role = kwargs.get('user_role', 'guest')
            if user_role != required_role:
                raise PermissionDeniedError(f"Требуется роль {required_role}, текущая роль {user_role}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

if __name__ == "__main__":
    pasta = DishFactory.create_dish(
        "maincourse",
        dish_id=1,
        name="Паста Карбонара",
        price=450,
        ingredients=["паста", "бекон", "яйца", "сливки", "пармезан"],
        calories=650
    )

    tiramisu = DishFactory.create_dish(
        "dessert",
        dish_id=2,
        name="Тирамису",
        price=280,
        ingredients=["маскарпоне", "печенье савоярди", "кофе", "какао"],
        sugar_content=25.5
    )

    lemonade = DishFactory.create_dish(
        "beverage",
        dish_id=3,
        name="Домашний лимонад",
        price=150,
        ingredients=["лимоны", "вода", "сахар", "мята"],
        volume=0.5
    )

    customer = Customer("Иван Иванов", "+79161234567", "ivan@example.com")
    order = Order(1, customer, "ул. Примерная, д. 10, кв. 5")

    order.add_dish(pasta)
    order.add_dish(tiramisu)
    order.add_dish(lemonade)

    print(order.generate_report())

    delivery_process = CourierDeliveryProcess()
    print(delivery_process.deliver_order(order))

    change_handler = CallCenterOperator(Manager(Admin()))
    change_request = {'change_type': 'price_change'}
    change_handler.handle_request(order, change_request)

    order_data = order.to_dict()
    with open('order.json', 'w') as f:
        json.dump(order_data, f, indent=2)

    with open('order.json') as f:
        loaded_order = Order.from_dict(json.load(f))
    print("\nЗагруженный заказ:")
    print(loaded_order.generate_report())

    @check_permissions('admin')
    def cancel_order(order_id: int, user_role: str):
        print(f"Заказ #{order_id} отменен")

    try:
        cancel_order(1, user_role='user')
    except PermissionDeniedError as e:
        print(f"Ошибка: {e}")


Notification: В заказ #1 добавлено блюдо: Паста Карбонара
Notification: В заказ #1 добавлено блюдо: Тирамису
Notification: В заказ #1 добавлено блюдо: Домашний лимонад
Отчет по заказу #1
Клиент: Иван Иванов
Телефон: +79161234567
Адрес доставки: ул. Примерная, д. 10, кв. 5
Блюда:
Основное блюдо: Паста Карбонара, Цена: 450, Калорийность: 650
Десерт: Тирамису, Цена: 280, Содержание сахара: 25.5g
Напиток: Домашний лимонад, Цена: 150, Объем: 0.5л
Общая стоимость: 925.0
Подготовка заказа #1 для курьерской доставки
Доставка заказа #1 курьером
Заказ #1 доставлен курьером
Менеджер одобрил изменение стоимости
Notification: В заказ #1 добавлено блюдо: Паста Карбонара
Notification: В заказ #1 добавлено блюдо: Тирамису
Notification: В заказ #1 добавлено блюдо: Домашний лимонад

Загруженный заказ:
Отчет по заказу #1
Клиент: Иван Иванов
Телефон: +79161234567
Адрес доставки: ул. Примерная, д. 10, кв. 5
Блюда:
Основное блюдо: Паста Карбонара, Цена: 450, Калорийность: 650
Десерт: Тирамису, Цена: 280, Со